In [ ]:
def basic_s(n):
    """The sum of all divisors of n (except n). Brute force method, checking every number up to n/2 to see if its a divisor."""
    
    #list of divisors
    divisors = []

    #checks every number up to n to see if they are divisors, and if they are, add to the divisors list
    for i in range(1, (n//2 + 1)):                     #used integer division for odd numbers to make sure that it's not a decimal point. 
        if (n % i) == 0:
            divisors.append(i)
            
    #returns the sum of all divisors of n, except for n itself.
    return sum(divisors)

def smallest_factor(n):
    """Returns the smallest factor of a positive integer n. Taken from number_theory_lecture_functions from lecture 8.2 Appendix."""
    sqrt=n**0.5
    i=2
    while i<=sqrt:
        if n%i==0:
            return i                            #If we get here, return i as the value.
        i+=1
    return n                                    #If we get through the whole while loop, return n.
    
def decompose(n):
    """Generates a dictionary representing the prime decomposition. Taken from number_theory_lecture_functions from lecture 8.2 Appendix."""
    factors={}
    current_number=n                            #divide current_number by the factor found found until it reaches 1
    while current_number > 1:
        p=smallest_factor(current_number)
        if p in factors.keys():                 #if p is not a new factor, increase the power
            factors[p]+=1
        else:
            factors[p]=1                        #if p is a new factor, create a new entry
        current_number = current_number//p
    return factors

def s(n):      
    """The sum of all divisors of n (except n). Takes advantage of the fact that the sum of all divisors is a multiplicative function"""

    #gives the prime decomposition of n.
    k = decompose(n)
    result = 1

    for p in k.keys():
        result *= (p**(k[p]+1)-1)//(p-1)            #Uses the formula for the geometric series of the sum of divisors where k[p] is the prime power.
        
    return result - n                               #gives the sum of divisors without n.

def a(n, iterations):
    """The aliquot sequence of s(n). Inputs are n (the starting number), and iterations (number of elements in the sequence)"""
    
    #the aliquot sequence of s(n) as a list, which is also the output.
    aliquot_seq = [n]
    
    for i in range(iterations - 1):

        sum_div = s(n)
        
        #if s(n) = 0 then the sequence terminates.
        if sum_div == 0: 
            aliquot_seq.append(sum_div)
            return aliquot_seq

        #if s(n) > 10^9 then the sequence terminates    
        elif sum_div > 10**9:
            return aliquot_seq

        #adds the current s(n) to the aliquot sequence and changes the input value to s(n)    
        else:
            aliquot_seq.append(sum_div)
            n = sum_div    
            
    return aliquot_seq